# Evaluate the FJLT reconstruction errors in DMD

In [ ]:
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator
import scipy.signal as sig
from tqdm import tqdm
import sys
sys.path.append('../../../utils')
import gc
from TurboFJLT import *
from InstantFrequency import inst_freq
from TurboFJLT_helpers import TurboHDF5Reader, TurboVisual

In [ ]:
%config InlineBackend.figure_format='retina'
cmap_b = mpl.colormaps['Blues']
cmap_r = mpl.colormaps['Reds']
blue = cmap_b(0.7)
red = 'r'
green = 'g'

# Synchronise this with dmd.ipynb
e = 0.01
modes_to_show = 20
num_snapshots = 300
snapshot_sequence = list(range(num_snapshots))

In [ ]:
Q_file = "../../../data/fine_airfoil_cascade.h5"
dir_dmd_file = "../data/direct_dmd.h5"
fjlt_dmd_file = "../data/fjlt_dmd_{}_linking_snapshots.h5"

In [ ]:
reader = TurboHDF5Reader(Q_file)
print(reader)

In [ ]:
with h5.File(dir_dmd_file, 'r') as f:
    dir_amps = f["/amps"][()]
dir_ordering = np.argsort(np.abs(dir_amps))[::-1]
dir_amps = dir_amps[dir_ordering]

### Plot eigenvalues and amplitudes for varying numbers of linking snapshots
The eigenvalues and amplitudes are ordered by descending amplitude

In [ ]:
plt.style.use("../../../mplstyles/paper_full.mplstyle")

In [ ]:
def plot_dmd_amplitudes(ax, a, a_f, label, index_a=None, index_a_f=None):
    index_a = np.arange(len(a)) if index_a is None else index_a
    index_a_f = np.arange(len(a_f)) if index_a_f is None else index_a_f
    ax.plot(index_a, a, color=blue, marker="o", label="Direct")
    ax.plot(index_a_f, a_f, color=red, marker="x", label="FJLT $N={}$".format(label))
    ax.set_xlim([0, max([len(a), len(a_f)])])
    ax.set_ylim([0, None])
    # ax.set_xlabel(r"Mode index (sorted by descending mode amplitude)")
    # ax.set_ylabel(r"$|a|$")
    ax.legend()
    return None

In [ ]:
num_linking_snapshots = [2, 4, 6, 8, 12, 16, 24, 32]
fig, axs = plt.subplots(ncols=2, nrows=4, sharex=True, sharey=True, figsize=(6.9, 3.5))
for n_sp, ax in zip(num_linking_snapshots, axs.flatten()):
    with h5.File(fjlt_dmd_file.format(n_sp), 'r') as f:
        fjlt_amps = f["/amps"][()]
        fjlt_ordering = np.argsort(np.abs(fjlt_amps))[::-1]
        fjlt_amps = fjlt_amps[fjlt_ordering]
        plot_dmd_amplitudes(ax,
                            np.abs(dir_amps),
                            np.abs(fjlt_amps),
                            label=n_sp)
    if np.any(ax==axs[:, 0]):
        ax.set_ylabel(r"$|a|$")
    if np.any(ax==axs[-1, :]):
        ax.set_xlabel(r"Mode index (sorted by descending mode amplitude)")
plt.savefig("../figures/dmd_amplitudes_all.pdf".format(n_sp),
            bbox_inches="tight", pad_inches=0.1,
            facecolor=None, edgecolor='auto'
            )

In [ ]:
plt.style.use("../../../mplstyles/paper_full_75pc.mplstyle")

In [ ]:
def plot_dmd_eigenvalues(eigs, eigs_f):
    circle = np.exp(1j*np.linspace(0, 2*np.pi, 200))

    fig, axs = plt.subplots(ncols=1, nrows=1)
    axs.plot(circle.real, circle.imag, color="k", linewidth=1, alpha=0.1)
    axs.scatter(eigs.real, eigs.imag, marker="o", color=blue)
    axs.scatter(eigs_f.real, eigs_f.imag, marker="x", color=red)

    axs.set_xlim(0, 1.1)
    axs.set_aspect("equal");
    axs.set_xlabel(r"Re$(\lambda)$")
    axs.set_xlabel(r"Im$(\lambda)$")
    return fig

In [ ]:
with h5.File(dir_dmd_file, 'r') as f:
    dir_eigs = f["/eigs"][()]
dir_eigs = dir_eigs[dir_ordering]

In [ ]:
def plot_dmd_eigenvalues_on_axis(ax, _eigs, _amps, _eigs_f, offset):
    amps = np.abs(_amps)
    cm_offset = 0.1
    amps_colour_scale = cm_offset + (amps-min(amps))/(max(amps)-min(amps))*(1-cm_offset)
    marker_colors = plt.cm.Blues(amps_colour_scale)
    edge_color = plt.cm.Blues(1.0)

    circle = np.exp(1j*np.linspace(-2*np.pi/5, 2*np.pi/5, 200))

    circle += offset
    eigs = _eigs + offset
    eigs_f = _eigs_f + offset

    ax.plot(circle.real, circle.imag, color="k", linewidth=1, alpha=0.2)
    ax.scatter(eigs.real, eigs.imag, marker="o", color=marker_colors, edgecolor=edge_color)
    ax.scatter(eigs_f.real, eigs_f.imag, marker="o", facecolors='none', edgecolor=red, linewidth=1)

    # ax.set_xlim(0, None)
    ax.set_aspect("equal");
    ax.set_ylim([-1, 1])
    ax.set_xlabel(r"Re$(\lambda)$")
    ax.set_ylabel(r"Im$(\lambda)$")
    ax.tick_params(
        axis='x',
        which='both',
        bottom=False,
        top=False,
        labelbottom=False)


Eigenvalues are plotted on the circle $|\lambda| = 1$. Blue eigenvalues denote those obtained from the DMD of $Q$, while the red are those obtained using the DMD on $B=FJLT(Q)$

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)

for i, n_sp in enumerate(num_linking_snapshots):
    with h5.File(fjlt_dmd_file.format(n_sp), 'r') as f:
        fjlt_amps = f["/amps"][()]
        fjlt_eigs = f["/eigs"][()]
    fjlt_ordering = np.argsort(np.abs(fjlt_amps))[::-1]
    fjlt_eigs = fjlt_eigs[fjlt_ordering]
    plot_dmd_eigenvalues_on_axis(ax, dir_eigs, dir_amps,
                                      fjlt_eigs, 0.5*i+0j)
    plt.savefig("../figures/dmd_eigenvalues.pdf",
        bbox_inches="tight", pad_inches=0.1,
        facecolor=None, edgecolor='auto',
       )


In [ ]:
def eigenvalue_error_analysis(eigs, eigs_f):
    err_eigs_f = np.zeros_like(eigs_f, dtype=float)
    dir_indices = []
    for i, eig_f in enumerate(eigs_f):
        matching_index = np.argmin(np.abs(eigs-eig_f))
        err_eigs_f[i] = np.abs(eigs[matching_index]-eig_f)/np.abs(eigs[matching_index])*100
        np.delete(eigs, matching_index)
        dir_indices.append(matching_index)
    return err_eigs_f, dir_indices

As there is no bijective correspondence between the arrays containing $\lambda_Q$ are $\lambda_B$, these are first ordered using ``dir_index_to_fjlt_eigenvalue[]``, before the comparison is made

In [ ]:
eigenvalue_errors = [np.zeros(len(dir_eigs), dtype="float") for _ in num_linking_snapshots]
direct_indices = [np.zeros(len(dir_eigs), dtype="int") for _ in num_linking_snapshots]
dir_index_to_fjlt_eigenvalue = {}
for i, n_sp in enumerate(num_linking_snapshots):
    with h5.File(fjlt_dmd_file.format(n_sp), 'r') as f:
        fjlt_amps = f["/amps"][()]
        fjlt_eigs = f["/eigs"][()]
    fjlt_ordering = np.argsort(np.abs(fjlt_amps))[::-1]
    fjlt_eigs = fjlt_eigs[fjlt_ordering]
    fjlt_amps = fjlt_amps[fjlt_ordering]
    err, inds = eigenvalue_error_analysis(dir_eigs,
                                          fjlt_eigs)
    eigenvalue_errors[i][inds] = err
    direct_indices[i][inds] = inds
    dir_index_to_fjlt_eigenvalue[n_sp] = {ind : (fjlt_eig, fjlt_amp) for ind, fjlt_eig, fjlt_amp in zip(inds, fjlt_eigs, fjlt_amps)}
    # Trim the zeros
    direct_indices[i] = direct_indices[i][eigenvalue_errors[i]!=0]
    eigenvalue_errors[i] = eigenvalue_errors[i][eigenvalue_errors[i]!=0]

In [ ]:
# Repeat the first figure but with correct ordering of the amplitudes to match direct modes
fig, axs = plt.subplots(ncols=2, nrows=4, sharex=True, sharey=True, figsize=(6.9, 3.5))
for i, (n_sp, ax) in enumerate(zip(num_linking_snapshots, axs.flatten())):
    with h5.File(fjlt_dmd_file.format(n_sp), 'r') as f:
        fjlt_amps = f["/amps"][()]
        fjlt_ordering = np.argsort(np.abs(fjlt_amps))[::-1]
        fjlt_amps = fjlt_amps[fjlt_ordering]
        plot_dmd_amplitudes(ax,
                            np.abs(dir_amps),
                            np.abs(fjlt_amps),
                            label=n_sp,
                            index_a_f=direct_indices[i])
    if np.any(ax==axs[:, 0]):
        ax.set_ylabel(r"$|a|$")
    if np.any(ax==axs[-1, :]):
        ax.set_xlabel(r"Mode index (sorted by descending mode amplitude)")
plt.savefig("../figures/dmd_amplitudes_all_ordered.pdf".format(n_sp),
            bbox_inches="tight", pad_inches=0.1,
            facecolor=None, edgecolor='auto'
            )

In [ ]:
plt.style.use("../../../mplstyles/paper_half.mplstyle")
colors = plt.cm.Blues(np.linspace(0.3,1,len(num_linking_snapshots)))

fig = plt.figure()
ax=fig.gca()
for i, n_sp in enumerate(num_linking_snapshots):
    ax.plot(direct_indices[i], eigenvalue_errors[i], marker="o", color=colors[i], label=r"${}$".format(n_sp))

ax.set_xlim([0, None])
ax.set_ylim([0, 3.0])
ax.set_xlabel("Mode index (sorted by descending mode amplitude)")
ax.set_ylabel("Relative mode error (%)")
ax.legend(ncols=2)
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.savefig("../figures/dmd_eigenvalue_errors.pdf",
        bbox_inches="tight", pad_inches=0.1,
        facecolor=None, edgecolor='auto',
       )

### Reconstruction of mode shape

Mode reconstruction is achieved by performing a DFT on the $Q-Q_{mf}$ time-series matrix, which has the meanflow removed from the snapshots. 
The result is scaled by $1/(N*amplitude)$ to complete the inversion.  
The kernel uses discrete freqiencies, by using the casting $k = int(\omega/\Delta\omega)$. However, note that generally $k \neq \omega/\Delta\omega$, or the continuous case.

In [ ]:
def FFT_DMD_reconstruction(Q, mode, amp):

    mode = mode/np.abs(mode)
    # Parameters
    N = Q.shape[1]
    dt = 1
    domega = 2*np.pi/N/dt
    omega = np.imag(np.log(mode))
    k = int(omega/domega)

    # Extraction of the shape function
    ker = np.exp(-2*np.pi*1j*k*np.arange(N)/N).T
    sh = Q.dot(ker)

    sh /= N * amp

    return sh

Mode reconstruction is handled by first processing the column vector and then reordering the elements of the vector into the two-dimensional geometry.

In [ ]:
def formQ(reader, seq_to_extract):
    q_mf = reader.load_meanflow()
    Q = reader.load_full(seq_to_extract)
    for i in range(Q.shape[1]):
        Q[:, i] -= q_mf
    return Q

In [ ]:
with h5.File(dir_dmd_file, 'r') as f:
    dir_modes = f["/modes"][()]
dir_modes = dir_modes[:, dir_ordering]
Q = formQ(reader, snapshot_sequence)

In [ ]:
vis_num_linking_snapshots = 6  # Which data series to visualise
num_modes_to_plot = 12
fjlt_eigenvalue_mapping_dictionary = dir_index_to_fjlt_eigenvalue[vis_num_linking_snapshots]

In [ ]:
dir_modes_reconstructed = np.array([FFT_DMD_reconstruction(Q, eig, amp) for eig, amp in tqdm(zip(dir_eigs[:num_modes_to_plot], dir_amps[:num_modes_to_plot]))]).T

In [ ]:
fjlt_modes_reconstructed = np.array([FFT_DMD_reconstruction(Q, fjlt_eigenvalue_mapping_dictionary[dir_ind][0], fjlt_eigenvalue_mapping_dictionary[dir_ind][1])
                                    if dir_ind in fjlt_eigenvalue_mapping_dictionary
                                    else None
                                    for dir_ind in tqdm(range(num_modes_to_plot))]).T

In [ ]:
del Q
gc.collect()

In [ ]:
tv = TurboVisual(reader)
dir_mode_vis = reader.reconstruct_field(dir_modes[:, :num_modes_to_plot])
dir_mode_reconstructed_vis = reader.reconstruct_field(dir_modes_reconstructed[:, :num_modes_to_plot])
fjlt_mode_reconstructed_vis = reader.reconstruct_field(fjlt_modes_reconstructed[:, :num_modes_to_plot])

In [ ]:
# for i, (dir_field, dir_reconstructed_field, fjlt_reconstructed_field) in enumerate(zip(dir_mode_vis, dir_mode_reconstructed_vis, fjlt_mode_reconstructed_vis)):

#     fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(6.9/2*3, 2), sharey=True)
#     vmin, vmax = tv.plot_field(axs[0], dir_field, 0, plot_type="real", centre_colourmap=True, region=0, cmap="RdBu_r")
#     _, _ = tv.plot_field(axs[1], dir_reconstructed_field, 0, plot_type="real", centre_colourmap=True, limits=(vmin, vmax), region=0, cmap="RdBu_r")
#     if fjlt_reconstructed_field is not None:
#         _, _ = tv.plot_field(axs[2], fjlt_reconstructed_field, 0, plot_type="real", centre_colourmap=True, limits=(vmin, vmax), region=0, cmap="RdBu_r")

#     axs[0].set_title("Mode from direct DMD")
#     axs[1].set_title("Reconst. mode from direct [BAD SCALING-DON'T PUBLISH]") # Bad scaling because just checking mode shape. Reconstruction needs proper amplitude inversion and scale factors for FFT
#     axs[2].set_title("Reconst. mode from FJLT [BAD SCALING-DON'T PUBLISH]")
#     # plt.savefig("../figures/DO_NOT_PUBLISH_dmd_modes_n_sp_{vis_num_linking_snapshots}_mode_index_{i}.pdf",
#     #     bbox_inches="tight", pad_inches=0.1,
#     #     facecolor=None, edgecolor='auto',
#     #    )

### Compare snapshots on single figure

In [ ]:
def visualise_field(ax, field, grid, vmin, vmax, h_pass, plot_offset):
    for i in range(3):
        for j in [-1, 0, 1]:
            ax.pcolormesh(grid[i][:, :, 0], grid[i][:, :, 1]+j*h_pass-plot_offset,
                        field[i],
                        vmin=vmin, vmax=vmax, shading="gouraud", cmap="RdBu_r", rasterized=True)
    return None

In [ ]:
def compare_snapshots(axs, field_A, field_B, turboreader, visualisation_variable, snapshot_index, plot_offset):
    field_A = [field_A[snapshot_index][i][:, :, visualisation_variable].real for i in range(turboreader.num_regions)]
    field_B = [field_B[snapshot_index][i][:, :, visualisation_variable].real for i in range(turboreader.num_regions)]

    # Multiply by ±1 to get the same field
    A_factor = 1 if field_A[0][0, 0] >= 0 else -1
    B_factor = 1 if field_B[0][0, 0] >= 0 else -1
    field_A = [A_factor*field_A[i] for i in range(turboreader.num_regions)]
    field_B = [B_factor*field_B[i] for i in range(turboreader.num_regions)]

    # vminA = np.min([np.min(region) for region in field_A])
    # vmaxA = np.max([np.max(region) for region in field_A])

    vminA = np.min(np.min(field_A[0]))
    vmaxA = np.max(np.max(field_A[0]))

    vminB = np.min(np.min(field_B[0]))
    vmaxB = np.max(np.max(field_B[0]))

    vmin = min([vminA, vminB])
    vmax = max([vmaxA, vmaxB])
    vabs = max([abs(vmin), abs(vmax)])


    grid = turboreader.load_grid()

    visualise_field(axs[0], field_A, grid, -vabs, vabs, 0.6, plot_offset)
    visualise_field(axs[1], field_B, grid, -vabs, vabs, 0.6, plot_offset)

    for ax in axs:
        ax.set_aspect("equal")

    for i in range(2):
        axs[i].set_xlabel(r"$x$")
    axs[0].set_ylabel(r"$y$")

    axs[0].set_title("Direct")
    axs[1].set_title("FJLT reconstructed")

    for ax in axs:
        ax.axis("off")

    return None

In [ ]:
def compare_three_snapshots(axs, field_A, field_B, field_C, turboreader, visualisation_variable, snapshot_index, plot_offset):
    field_A = [field_A[snapshot_index][i][:, :, visualisation_variable] for i in range(turboreader.num_regions)]
    field_B = [field_B[snapshot_index][i][:, :, visualisation_variable] for i in range(turboreader.num_regions)]
    field_C = [field_C[snapshot_index][i][:, :, visualisation_variable] for i in range(turboreader.num_regions)]

    # # Multiply by ±1 to get the same field
    # A_factor = 1 if field_A[0][0, 0] >= 0 else -1
    # B_factor = 1 if field_B[0][0, 0] >= 0 else -1
    # C_factor = 1 if field_C[0][0, 0] >= 0 else -1

    # Divide out the phase from the field
    A_factor = np.exp(-1j*np.angle(field_A[1][0, 0]))
    B_factor = np.exp(-1j*np.angle(field_B[1][0, 0]))
    C_factor = np.exp(-1j*np.angle(field_C[1][0, 0]))

    field_A = [(A_factor*field_A[i]).real for i in range(turboreader.num_regions)]
    field_B = [(B_factor*field_B[i]).real for i in range(turboreader.num_regions)]
    field_C = [(C_factor*field_C[i]).real for i in range(turboreader.num_regions)]

    # vminA = np.min([np.min(region) for region in field_A])
    # vmaxA = np.max([np.max(region) for region in field_A])

    vminA = np.min(np.min(field_A[0]))
    vmaxA = np.max(np.max(field_A[0]))
    vabsA = max([abs(vminA), abs(vmaxA)])

    vminB = np.min(np.min(field_B[0]))
    vmaxB = np.max(np.max(field_B[0]))
    vabsB = max([abs(vminB), abs(vmaxB)])

    vminC = np.min(np.min(field_C[0]))
    vmaxC = np.max(np.max(field_C[0]))
    vabsC = max([abs(vminC), abs(vmaxC)])

    # vmin = min([vminA, vminB, vminC])
    # vmax = max([vmaxA, vmaxB, vmaxC])
    # vabs = max([abs(vmin), abs(vmax)])


    grid = turboreader.load_grid()

    visualise_field(axs[0], field_A, grid, -vabsA, vabsA, 0.6, plot_offset)
    visualise_field(axs[1], field_B, grid, -vabsB, vabsB, 0.6, plot_offset)
    visualise_field(axs[2], field_C, grid, -vabsC, vabsC, 0.6, plot_offset)

    for ax in axs:
        ax.set_aspect("equal")

    for i in range(3):
        axs[i].set_xlabel(r"$x$")
    axs[0].set_ylabel(r"$y$")

    axs[0].set_title("Direct")
    axs[1].set_title("Direct reconstructed")
    axs[2].set_title("FJLT reconstructed")

    for ax in axs:
        ax.axis("off")

    return None

In [ ]:
# Need to plot every second one as they come in c.c. pairs
dir_mode_vis_cut = []
dir_mode_reconstructed_vis_cut = []
fjlt_mode_reconstructed_vis_cut = []

for i, (dir_field, dir_reconstr_field, fjlt_reconstr_field) in enumerate(zip(dir_mode_vis, dir_mode_reconstructed_vis, fjlt_mode_reconstructed_vis)):
    if i%2 == 0:
        dir_mode_vis_cut.append(dir_field)
        dir_mode_reconstructed_vis_cut.append(dir_reconstr_field)
        fjlt_mode_reconstructed_vis_cut.append(fjlt_reconstr_field)

In [ ]:
plt.style.use("../../../mplstyles/paper_full.mplstyle")
plot_offset = 0.6*3 + 0.3

fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(5.18, 2*8), sharey=True)

for mode in range(6):
    compare_three_snapshots(axs, dir_mode_vis_cut, dir_mode_reconstructed_vis_cut, fjlt_mode_reconstructed_vis_cut, reader, 0, mode, plot_offset*mode)
plt.savefig("../figures/dmd_reconst_num_snaps_{}.pdf".format(num_snapshots),
            bbox_inches="tight", pad_inches=0.1,
            facecolor=None, edgecolor='auto', dpi=300
            )